#Installs and utils

In [1]:
!pip install transformers
!pip install SentencePiece
!pip install tensorflow_addons
!pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from bertviz import model_view
from transformers import utils
utils.logging.set_verbosity_error()
import shutil, sys

In [5]:
import torch

In [6]:
from transformers import AutoModel, AutoTokenizer
from transformers import XLNetModel, XLNetTokenizer

#Data Loading and seeds

In [8]:
data=pd.read_csv('/content/drive/MyDrive/wellness/MultiLabel_WD - Sheet1 (1) (1).csv')

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_List].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [10]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [11]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [12]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [13]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs,_ = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs,targets)

        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs,_ = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs,targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

#6-label

In [14]:
data=data.astype({'Spiritual':'float', 'Physical':'float', 'Intellectual':'float', 'Social':'float', 'Vocational':'float',
       'Emotional':'float'})

In [15]:
target_List = ['Spiritual', 'Physical', 'Intellectual', 'Social', 'Vocational','Emotional']

In [16]:
MAX_LEN = 64
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE= 1e-05
dimension=6

In [17]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=345).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [18]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=546).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [19]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=200).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [20]:
import tensorflow_addons as tfa
metric = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=6)
metric.update_state(val_list,final_list)
result = metric.result()
result.numpy()

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


NameError: ignored

In [ ]:
val = pd.DataFrame(val_list, columns = target_List)

In [ ]:
fin = pd.DataFrame(final_list, columns = target_List)

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Physical"],fin["Physical"])

In [ ]:
target_List

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Spiritual"],fin["Spiritual"])

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Intellectual"],fin["Intellectual"])

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Social"],fin["Social"])

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Vocational"],fin["Vocational"])

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Emotional"],fin["Emotional"])

#5-label

In [ ]:
data=data.astype({'Spiritual':'int32', 'Physical':'int32', 'Intellectual':'int32', 'Social':'int32', 'Vocational':'int32',
       'Emotional':'int32'})

In [ ]:
data ['Spiritual or Emotional'] = data.Spiritual | data.Emotional

In [ ]:
data = data.drop(['Spiritual', 'Emotional'], axis=1)

In [ ]:
data=data.astype({ 'Physical':'float', 'Intellectual':'float', 'Social':'float', 'Vocational':'float',
       'Spiritual or Emotional':'float'})

In [ ]:
target_List = ['Physical', 'Intellectual', 'Social', 'Vocational','Spiritual or Emotional']

In [ ]:
MAX_LEN = 64
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE= 1e-05
dimension=5

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=345).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=546).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=678).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
import tensorflow_addons as tfa
metric = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
metric.update_state(val_list,final_list)
result = metric.result()
result.numpy()

0.8675862

In [ ]:
val = pd.DataFrame(val_list, columns = target_List)

In [ ]:
fin = pd.DataFrame(final_list, columns = target_List)

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Physical"],fin["Physical"])

0.8775653717590278

In [ ]:
target_List

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Spiritual or Emotional"],fin["Spiritual or Emotional"])

0.8402778941387434

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Intellectual"],fin["Intellectual"])

0.8932728260476415

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Social"],fin["Social"])

0.8731963769142771

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Vocational"],fin["Vocational"])

0.9035843974425796

#4-label

In [ ]:
data=data.astype({ 'Physical':'float', 'Intellectual':'float', 'Social':'float', 'Vocational':'float',
       'Spiritual or Emotional':'float'})

In [ ]:
data=data.astype({'Spiritual or Emotional':'int32', 'Physical':'int32', 'Intellectual':'int32', 'Social':'int32', 'Vocational':'int32',
       })

In [ ]:
data ['Spiritual or Emotional or Physical'] = data ['Spiritual or Emotional'] | data['Physical']

In [ ]:
data = data.drop(['Physical'], axis=1)

In [ ]:
data=data.drop(['Spiritual or Emotional'], axis=1)

In [ ]:
data=data.astype({  'Intellectual':'float', 'Social':'float', 'Vocational':'float',
       'Spiritual or Emotional or Physical':'float'})

In [ ]:
target_List = ['Intellectual', 'Social', 'Vocational','Spiritual or Emotional or Physical']

In [ ]:
MAX_LEN = 64
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE= 1e-05
dimension=4

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=345).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=546).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=678).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
import tensorflow_addons as tfa
metric = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=4)
metric.update_state(val_list,final_list)
result = metric.result()
result.numpy()

In [ ]:
val = pd.DataFrame(val_list, columns = target_List)

In [ ]:
fin = pd.DataFrame(final_list, columns = target_List)

In [ ]:
target_List

['Intellectual', 'Social', 'Vocational', 'Spiritual or Emotional or Physical']

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Spiritual or Emotional or Physical"],fin["Spiritual or Emotional or Physical"])

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Intellectual"],fin["Intellectual"])

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Social"],fin["Social"])

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Vocational"],fin["Vocational"])

#3-label

In [ ]:
data=data.astype({'Spiritual or Emotional or Physical':'int32', 'Intellectual':'int32', 'Social':'int32', 'Vocational':'int32',
       })

In [ ]:
data ['Intellectual or Vocational'] = data ['Intellectual'] | data['Vocational']

In [ ]:
data = data.drop(['Intellectual', 'Vocational'], axis=1)

In [ ]:
data=data.astype({  'Intellectual or Vocational':'float', 'Social':'float',
       'Spiritual or Emotional or Physical':'float'})

In [ ]:
target_List = ['Social', 'Spiritual or Emotional or Physical',
       'Intellectual or Vocational']

In [ ]:
MAX_LEN = 32
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE= 1e-05
dimension=3

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=345).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=546).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
train_size = 0.8
train_df = data.sample(frac=train_size, random_state=678).reset_index (drop=True)
val_df = data.drop (train_df.index).reset_index (drop=True)

In [ ]:
import tensorflow_addons as tfa
metric = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=3)
metric.update_state(val_list,final_list)
result = metric.result()
result.numpy()

0.9113044

In [ ]:
val = pd.DataFrame(val_list, columns = target_List)

In [ ]:
fin = pd.DataFrame(final_list, columns = target_List)

In [ ]:
target_List

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Spiritual or Emotional or Physical"],fin["Spiritual or Emotional or Physical"])

0.9000066192215204

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Intellectual or Vocational"],fin["Intellectual or Vocational"])

0.9368273427127233

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(val["Social"],fin["Social"])

0.9122815768194011

#ERNIE

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nghuyong/ernie-2.0-en')

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class ERNIEClass(torch.nn.Module):
    def __init__(self):
        super(ERNIEClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('nghuyong/ernie-2.0-en')
        self.ernie_model = AutoModel.from_pretrained('nghuyong/ernie-2.0-en', output_attentions=True, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output_dict = self.ernie_model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True,
            return_dict=True
        )
        last_hidden_state = output_dict.last_hidden_state
        attention_weights = output_dict.attentions
        output_dropout = self.dropout(last_hidden_state[:, -1, :])
        output = self.linear(output_dropout)
        return output, attention_weights[-1]

model = ERNIEClass()
model.to(device)

ERNIEClass(
  (ernie_model): ErnieModel(
    (embeddings): ErnieEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(4, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ErnieEncoder(
      (layer): ModuleList(
        (0-11): 12 x ErnieLayer(
          (attention): ErnieAttention(
            (self): ErnieSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ErnieSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, elem

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
report=classification_report(val_list, final_list,target_names=label_names)

In [ ]:
print(report)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

#XLNET

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class XLNetClass(torch.nn.Module):
    def __init__(self):
        super(XLNetClass, self).__init__()
        self.xlnet_model = XLNetModel.from_pretrained('xlnet-base-cased', output_attentions=True, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output_dict = self.xlnet_model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True,
            return_dict=True
        )
        last_hidden_state = output_dict.last_hidden_state
        attention_weights = output_dict.attentions
        output_dropout = self.dropout(last_hidden_state[:, -1, :])
        output = self.linear(output_dropout)
        return output, attention_weights[-1]

model = XLNetClass()
model.to(device)

XLNetClass(
  (xlnet_model): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (linear): Linear(in_features=768, out_features=4, bias=True)
)

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.003078 	Average Validation Loss: 0.009710
Validation loss decreased (inf --> 0.009710).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.002631 	Average Validation Loss: 0.007177
Validation loss decreased (0.009710 --> 0.007177).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      final_list.append(final_output[0])

In [ ]:
val_list=[]

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

#PsychBERT

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('mnaylor/psychbert-cased')

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class PsychBERTClass(torch.nn.Module):
    def __init__(self):
        super(PsychBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
        self.model = AutoModel.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output, output_with_attention


model = PsychBERTClass()
model.to(device)

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.001705 	Average Validation Loss: 0.004049
Validation loss decreased (inf --> 0.004049).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.001351 	Average Validation Loss: 0.003461
Validation loss decreased (0.004049 --> 0.003461).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

[[0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 0.0, 0.0

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

                        precision    recall  f1-score   support

              Physical       0.99      0.98      0.99       190
          Intellectual       0.98      0.99      0.99       127
                Social       1.00      1.00      1.00       440
            Vocational       1.00      0.99      1.00       115
Spiritual or Emotional       0.99      1.00      1.00       300

             micro avg       0.99      0.99      0.99      1172
             macro avg       0.99      0.99      0.99      1172
          weighted avg       0.99      0.99      0.99      1172
           samples avg       0.99      0.99      0.99      1172



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.9908536585365854


#ClinicalBERT

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class ClinicalBIGBERTClass(torch.nn.Module):
    def __init__(self):
        super(ClinicalBIGBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
        self.model = AutoModel.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention



model = ClinicalBIGBERTClass()
model.to(device)

ClinicalBIGBERTClass(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.001493 	Average Validation Loss: 0.003591
Validation loss decreased (inf --> 0.003591).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.001221 	Average Validation Loss: 0.002657
Validation loss decreased (0.003591 --> 0.002657).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

[[0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 0.0, 0.0

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

                        precision    recall  f1-score   support

              Physical       0.99      1.00      1.00       190
          Intellectual       0.98      1.00      0.99       127
                Social       1.00      1.00      1.00       440
            Vocational       1.00      0.98      0.99       115
Spiritual or Emotional       1.00      1.00      1.00       300

             micro avg       1.00      1.00      1.00      1172
             macro avg       1.00      1.00      1.00      1172
          weighted avg       1.00      1.00      1.00      1172
           samples avg       0.99      0.99      0.99      1172



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.9939024390243902


#MentalBERT

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
tokenizer=AutoTokenizer.from_pretrained('mental/mental-bert-base-uncased',use_auth_token=True)

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class MentalBERTClass(torch.nn.Module):
    def __init__(self):
        super(MentalBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('mental/mental-bert-base-uncased')
        self.model = AutoModel.from_pretrained('mental/mental-bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention



model = MentalBERTClass()
model.to(device)

MentalBERTClass(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.007384 	Average Validation Loss: 0.027407
Validation loss decreased (inf --> 0.027407).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.006541 	Average Validation Loss: 0.025243
Validation loss decreased (0.027407 --> 0.025243).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

                                    precision    recall  f1-score   support

                            Social       0.98      0.96      0.97       440
Spiritual or Emotional or Physical       0.95      0.98      0.97       422
        Intellectual or Vocational       0.96      0.95      0.96       210

                         micro avg       0.97      0.97      0.97      1072
                         macro avg       0.96      0.96      0.96      1072
                      weighted avg       0.97      0.97      0.97      1072
                       samples avg       0.95      0.95      0.95      1072



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.9435975609756098


#BERT

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class MentalBERTClass(torch.nn.Module):
    def __init__(self):
        super(MentalBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention



model = MentalBERTClass()
model.to(device)

MentalBERTClass(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.002309 	Average Validation Loss: 0.006267
Validation loss decreased (inf --> 0.006267).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.001634 	Average Validation Loss: 0.005346
Validation loss decreased (0.006267 --> 0.005346).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

[[0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 0.0, 0.0

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

                        precision    recall  f1-score   support

              Physical       0.99      1.00      1.00       190
          Intellectual       0.98      0.98      0.98       127
                Social       1.00      0.99      1.00       440
            Vocational       1.00      0.99      1.00       115
Spiritual or Emotional       0.99      0.99      0.99       300

             micro avg       0.99      0.99      0.99      1172
             macro avg       0.99      0.99      0.99      1172
          weighted avg       0.99      0.99      0.99      1172
           samples avg       0.99      0.99      0.99      1172



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.9862804878048781


#roBERTa

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class roBERTaClass(torch.nn.Module):
    def __init__(self):
        super(roBERTaClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained("roberta-base")
        self.model = AutoModel.from_pretrained("roberta-base", return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention



model = roBERTaClass()
model.to(device)

roBERTaClass(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.006928 	Average Validation Loss: 0.022692
Validation loss decreased (inf --> 0.022692).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.005537 	Average Validation Loss: 0.020808
Validation loss decreased (0.022692 --> 0.020808).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############


In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))